# __Regressions & Plotting__

In [5]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
#CSV version
# Price_Patent_Reg = pd.read_csv('data/Price_Patent_Reg.csv', index_col = 0, dtype = data_types)

In [9]:
import dill
Price_Patent_Reg = dill.load(open('data/features_created.pkd', 'rb'))
Price_Patent_Reg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 214082 entries, 68472 to 6411864
Data columns (total 15 columns):
nadac_per_unit                                214082 non-null float32
ndc                                           214082 non-null float32
days_before_patent_expires                    214082 non-null float32
generic_exists                                214082 non-null int32
effective_date_year                           214082 non-null float16
effective_date_month                          214082 non-null float16
effective_date_day                            214082 non-null float16
corresponding_generic_drug_effective_year     214082 non-null float16
corresponding_generic_drug_effective_month    214082 non-null float16
corresponding_generic_drug_effective_day      214082 non-null float16
dosage_form_SPRAY                             214082 non-null uint8
dosage_form_TABLET                            214082 non-null uint8
dosage_form_TABLET, CHEWABLE                  2140

### __Create test & train datasets__
From the dependent and independent variable dataframes

In [10]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(Price_Patent_Reg,
                                         test_size = 0.2,
                                         random_state = 1,
#                                          shuffle = True
                                        )    #shuffle data to avoid correlation to the natural order of the data

### __Create a class for estimating by group__

In [11]:
from sklearn import base
import numpy as np 
import pandas as pd

class GroupbyEstimator(base.BaseEstimator, base.RegressorMixin):

    def __init__(self, groupby_column, pipeline_factory):
        # column is the value to group by; estimator_factory can be called to produce estimators
        self.groupby_column = groupby_column
        self.pipeline_factory = pipeline_factory

    
    def fit(self, dataframe, label):
        # Create an estimator and fit it with the portion in each group (create and fit a model per city
        self.drugs_dict = {}
        self.label = label
        self.coefs_dict = {} 
        self.intercepts_dict = {} 
       
        dataframe = pd.get_dummies(dataframe)  #onehot encoder had problems with the data, so I'm getting the dummies with pandas here
        
        for name, values in dataframe.groupby(self.groupby_column):
            y = values[label]
            X = values.drop(columns = [label, self.groupby_column], axis = 1)
            self.drugs_dict[name] = self.pipeline_factory().fit(X, y)
            self.coefs_dict[name] = self.drugs_dict[name].named_steps["lin_reg"].coef_
            self.intercepts_dict[name] = self.drugs_dict[name].named_steps["lin_reg"].intercept_
        return self

    #Method to get the coefficients for each regression
    def get_coefs(self):       
        return self.coefs_dict
    
    #Method to get the intercepts for each regression
    def get_intercepts(self):
        return self.intercepts_dict
    
        
    def predict(self, test_data):
        price_pred_list = []

        for idx, row in test_data.iterrows():
            name = row[self.groupby_column]                                 #get drug name from drug column
            regression_coefs = self.drugs_dict[name]                        #get coefficients from fitting in drugs_dict
            row = pd.DataFrame(row).T
            X = row.drop(columns = [self.label, self.groupby_column], axis = 1).values.reshape(1, -1) #Drop ndc and price cols          

            drug_price_pred = regression_coefs.predict(X)    
            price_pred_list.append([name, drug_price_pred])
        return price_pred_list

### __Run Linear Regression__

In [12]:
from sklearn import base
import numpy as np 

def pipeline_factory():
    from sklearn.pipeline import Pipeline
    from sklearn.linear_model import LinearRegression

    return Pipeline([
                     ('lin_reg', LinearRegression())
                    ])

lin_model = GroupbyEstimator('ndc', pipeline_factory).fit(train_data,'nadac_per_unit')

In [13]:
# Predict & save results
results = lin_model.predict(test_data)
predictions = [x[1][0] for x in results]
actual = test_data.iloc[:,0]

### __Calculating scoring metrics__

And an explanation of each score metric being considered

In [14]:
from sklearn.metrics import explained_variance_score, max_error, mean_absolute_error, mean_squared_error, mean_squared_log_error, median_absolute_error, r2_score 

scoring_methods = [
                   explained_variance_score, # 1-(Var(predicted-true)/Var(true)); equal to R2 if mean(error) == 0 (e.g. true == 0)
                   max_error,                # captures the worst case error(residual) between the predicted value and the true value
                   mean_absolute_error,      # average of (the absolute value of) all residuals; less sensitive to outliers; lower is better
                   mean_squared_error,       # penalty for making more predictions varying from the actual value; more sensitive to outliers
                   mean_squared_log_error,   # treats small differences between small true and predicted differences the same as big differences between large true and predicted values
                   median_absolute_error,    # Robust (insensitive) to outliers
                   r2_score                  # The proportion of variance of the dependent variable that has been explained by the independent variables (multioutput param defaults to 'uniform_average')
                  ]

for method in scoring_methods:
    try: 
        score = method(actual, predictions)
        print(method, ': ', score)
    except ValueError:
        pass

<function explained_variance_score at 0x00000226D764EE58> :  0.991957870772865
<function max_error at 0x00000226D764EF78> :  6.771544137277715
<function mean_absolute_error at 0x00000226D764EC18> :  0.34681015310872365
<function mean_squared_error at 0x00000226D764ECA8> :  0.5272986145175887
<function median_absolute_error at 0x00000226D764EDC8> :  0.11239640688711461
<function r2_score at 0x00000226D764EEE8> :  0.9919577522368749


In [15]:
# Prediction/Actual values in dataframe for comparison (index is NDC number)
prediction_values = pd.DataFrame({'actual':actual, 'predictions': predictions})
prediction_values.head()

,actual,predictions
2509598,0.05078,0.051172
3532416,0.03889,0.457530
1726471,2.16548,2.281536
438940,0.10550,0.093438
5912535,1.01817,1.038753


### __Prep data for plotting__

In [16]:
def format_data(dataframe, filename, test = False):
    #change columns to datetime
    dataframe.loc[:, 'ndc'] = dataframe.loc[:, 'ndc'].astype('int64') #needed to convert int32 to int64 to hold larger number
    if test:
        dataframe.loc[:, ['effective_date_year', 'effective_date_month', 'effective_date_day']] = dataframe.loc[:, ['effective_date_year', 'effective_date_month', 'effective_date_day']].astype(str)
        dataframe.rename(columns = {'effective_date_year':'year', 'effective_date_month':'month', 'effective_date_day':'day'}, inplace = True)
        dataframe.loc[:, 'date'] = pd.to_datetime(dataframe[['year', 'month', 'day']], format = '%Y-%m-%d')
        dataframe.rename({'year':'effective_date_year', 'month':'effective_date_month', 'day':'effective_date_day'}, inplace = True)
        dataframe.loc[:, ['year', 'month', 'day']] = dataframe.loc[:, ['year', 'month', 'day']].astype(float).astype(int)
        dataframe.sort_values(['ndc', 'date'])
    else:
        dataframe.rename(columns = {'effective_date_year': 'year', 'effective_date_month': 'month', 'effective_date_day': 'day'}, inplace = True)

    #Keep only unique values
    dataframe.loc[:, 'year'] = dataframe.loc[:, 'year'].astype(int)
    dataframe.loc[:, 'month'] = dataframe.loc[:, 'month'].astype(int)
    dataframe.loc[:, 'day'] = dataframe.loc[:, 'day'].astype(int)

    return dataframe

#Save formatted data as follows
historical_data = format_data(train_data, 'historical_data', test = True)
prediction_data = format_data(test_data, 'pred_data')

C:\Users\Lofgran\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\Users\Lofgran\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)
C:\Users\Lofgran\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

## __Plot Drug Prices and Forecasts__

* Note that the following plot is still in development.  
* The current _working_ version (`Drug_Price_Plots.py` can be run from the command line via `bokeh serve --show Drug_Price_Plots.py`

In [17]:
#Plotting session
from bokeh.io import curdoc
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, Select, DataRange1d, HoverTool
from bokeh.plotting import figure

# Set up initial data
historical_data = historical_data.loc[:, ['ndc', 'date', 'nadac_per_unit']]
hist_temp = historical_data[historical_data.loc[:, 'ndc']==781593600].sort_values('date')
historical_source = ColumnDataSource(data = hist_temp)

#
import datetime as dt
#Get initial prediction
date = dt.datetime.strptime('-'.join(('2020', '3', '31')), '%Y-%m-%d')
new_prediction_data = prediction_data[prediction_data.loc[:, 'ndc']==781593600] #working
new_prediction_data.loc[:, 'year'] = date.year
new_prediction_data.loc[:, 'month'] = date.month
new_prediction_data.loc[:, 'day'] = date.day
new_prediction_data = lin_model.predict(new_prediction_data)
new_prediction_data = pd.DataFrame(data = {'ndc':new_prediction_data[0][0], 'nadac_per_unit':new_prediction_data[0][1][0]}, index = [0]) #these element slices are correct
new_prediction_data['date'] = pd.to_datetime(date, format='%Y-%m-%d')
new_prediction_data['ndc'] = new_prediction_data['ndc'].astype(float).astype('int64')
new_prediction_data['nadac_per_unit'] = new_prediction_data['nadac_per_unit'].astype('float16')
prediction_source = ColumnDataSource(data=new_prediction_data)

id_list = list(prediction_data['ndc'].astype(str))
# Set up plot
plot = figure(plot_height=800, plot_width=800, title='Drug Price Over Time',
              x_axis_type = 'datetime',
              tools="crosshair, pan, reset, save, wheel_zoom")
plot.xaxis.axis_label = 'Time'
plot.yaxis.axis_label = 'Price ($)'
plot.axis.axis_label_text_font_style = 'bold'
plot.grid.grid_line_alpha = 0.8
plot.title.text_font_size = '16pt'
plot.x_range = DataRange1d(range_padding = .01)
plot.add_tools(HoverTool(tooltips=[('Date', '@date{%F}'), ('Price', '@nadac_per_unit')],
                                    formatters = {'date': 'datetime'}))

plot.line('date', 'nadac_per_unit', source=historical_source, legend='Historical Price')
plot.scatter('date', 'nadac_per_unit', source=prediction_source, fill_color='red', size=8, legend='Predicted Price')

# Set up widgets
id_select = Select(title='Select a Drug ID Number', value='781593600', options=id_list)

# Set up callbacks
def update_data(attrname, old, new):

    #Get the current select value
    curr_id = id_select.value
    # Generate the new data
    new_historical = historical_data[historical_data.loc[:, 'ndc']==int(curr_id)]
    new_historical = new_historical.sort_values('date')

    new_prediction_data = prediction_data[prediction_data.loc[:, 'ndc']==int(curr_id)] #working
    date = dt.datetime.strptime('-'.join(('2020', '3', '31')), '%Y-%m-%d')
    new_prediction_data.loc[:, 'year'] = date.year
    new_prediction_data.loc[:, 'month'] = date.month
    new_prediction_data.loc[:, 'day'] = date.day
    new_prediction_data = lin_model.predict(new_prediction_data)
    new_prediction_data = pd.DataFrame(data = {'ndc':new_prediction_data[0][0], 'nadac_per_unit':new_prediction_data[0][1][0]}, index = [0]) #these element slices are correct
    new_prediction_data['date'] = pd.to_datetime(date, format='%Y-%m-%d')
    new_prediction_data['ndc'] = new_prediction_data['ndc'].astype(float).astype('int64')

    # Overwrite current data with new data
    historical_source.data = ColumnDataSource.from_df(new_historical)
    prediction_source.data = ColumnDataSource.from_df(new_prediction_data)

# Action when select menu changes
id_select.on_change('value', update_data)

# Set up layouts and add to document
inputs = column(id_select)

curdoc().add_root(row(inputs, plot, width = 1000))
curdoc().title = 'Drug Price Predictor'


C:\Users\Lofgran\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


# NO LONGER NEEDED?

### __Retrieve coefficients & intercepts for modeling__

In [27]:
# Retrieve all model coefficients 
all_model_coefs = lin_model.get_coefs()

# Retrieve all model intercepts 
all_model_intercepts = lin_model.get_intercepts()

#Convert coefficients to dataframe
train_data = train_data.drop(columns='date')
col_names = train_data.iloc[:,1:].columns
fit_details = pd.DataFrame(all_model_coefs).transpose()
fit_details.reset_index(inplace = True)


fit_details.columns = col_names

#Incorporate intercepts to dataframe
intercepts = pd.DataFrame(all_model_intercepts, index = ['intercepts']).transpose()
fit_details = fit_details.merge(intercepts, left_on = 'ndc', right_on = intercepts.index)

# Send final data to CSV
fit_details.to_csv('data/fit_details.csv')
# fit_details.head()

In [28]:
fit_details.head()

,ndc,days_before_patent_expires,generic_exists,year,month,day,corresponding_generic_drug_effective_year,corresponding_generic_drug_effective_month,corresponding_generic_drug_effective_day,dosage_form_SPRAY,dosage_form_TABLET,"dosage_form_TABLET, CHEWABLE","dosage_form_TABLET, EXTENDED RELEASE","dosage_form_TABLET, ORALLY DISINTEGRATING",intercepts
0,54034808.0,0.0,-1.387779e-16,-0.538991,-0.028986,-0.001299,0.0,0.0,0.0,0.0,0.230670,0.0,0.000000,0.0,1090.666712
1,54034820.0,0.0,-1.387779e-17,-0.534307,-0.025493,-0.001998,0.0,0.0,0.0,0.0,0.203701,0.0,0.000000,0.0,1081.237222
2,54318544.0,0.0,-1.734723e-18,0.004017,-0.004368,0.000033,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,-7.153616
3,93010904.0,0.0,1.301043e-18,0.011797,0.005596,-0.000012,0.0,0.0,0.0,0.0,0.000000,0.0,-0.217358,0.0,-23.310277
4,93010912.0,0.0,3.903128e-17,0.011525,0.005486,-0.000030,0.0,0.0,0.0,0.0,0.000000,0.0,-0.217925,0.0,-22.761722
